In [1]:
import geopandas as gpd
import pandas as pd
import os,re,json
import numpy as np
from p_tqdm import p_map

In [ ]:

# pip install mbutil

In [56]:
# lets generate the overview pictures 
#
floc = "tmp/county.geojson"
gpd.read_file('data/CountiesUA.geojson')[['OBJECTID', 'CTYUA20CD', 'CTYUA20NM','LONG', 'LAT', 'geometry']].to_file(floc, driver="GeoJSON")

os.system('tippecanoe -Z0 -z6 -f -o tiles/countries.mbtiles --coalesce-densest-as-needed --generate-id '+ floc)


For layer 0, using name "county"
216 features, 1304937 bytes of geometry, 16 bytes of separate metadata, 10662 bytes of string pool
  99.5%  6/31/19  


0

In [14]:
# #  reduce precision fun
# def around(coords, precision=5):
#     result = []
#     try:
#         return round(coords, precision)
#     except TypeError:
#         for coord in coords:
#             result.append(around(coord, precision))
#     return result

# from shapely.geometry import shape, mapping

# def layer_precision(geometry, precision=5):
#     geojson = mapping(geometry)
#     geojson['coordinates'] = around(geojson['coordinates'],precision)
#     return shape(geojson)


# gdf.loc[:,"geometry"] =  gdf["geometry"].apply(lambda x: layer_precision(x, precision=5))

In [60]:
base  = gpd.read_file('data/OA2011.geojson')[['OA11CD', 'LAD11CD',
       'geometry']]
centroids = gpd.read_file('data/centroids2011.geojson')[['lsoa01cd', 'geometry']]

base.to_file("tmp/OA11.geojson", driver="GeoJSON")
centroids.to_file("tmp/centroids11.geojson", driver="GeoJSON")

os.system('tippecanoe -Z6 -z9 -f -o tiles/area_centroids_6_9.mbtiles --coalesce-densest-as-needed --generate-id ./tmp/OA%d.geojson ./tmp/centroids%d.geojson'%(11,11))

# tippecanoe -o tiles/test.mbtiles -Z9 -z10 -f ./data/tile_OA11.geojson ./data/tile_centroids11.geojson


# gdf_base = gdf_base.append(df,ignore_index=True)
# gdf_base .to_file("myfile.gpkg", layer="layer1", driver="GPKG")

For layer 0, using name "OA11"
For layer 1, using name "centroids11"
215786 features, 19728264 bytes of geometry, 16 bytes of separate metadata, 2377501 bytes of string pool
tile 6/32/21 size is 559340 with detail 12, >500000    
Going to try keeping the sparsest 80.45% of the features to make it fit
tile 6/32/21 size is 534418 with detail 12, >500000    
Going to try keeping the sparsest 67.74% of the features to make it fit
tile 6/31/20 size is 1672552 with detail 12, >500000    
Going to try keeping the sparsest 26.90% of the features to make it fit
tile 6/32/21 size is 502263 with detail 12, >500000    
Going to try keeping the sparsest 60.69% of the features to make it fit
tile 6/31/21 size is 1749714 with detail 12, >500000    
Going to try keeping the sparsest 25.72% of the features to make it fit
tile 6/31/20 size is 954993 with detail 12, >500000    
Going to try keeping the sparsest 12.68% of the features to make it fit
tile 6/31/21 size is 1025425 with detail 12, >500000    

0

In [ ]:
# base  = gpd.read_file('data/OA2011.geojson')[['OA01CD', 'LAD01CD',
#        'geometry']]
# centroids = gpd.read_file('data/centroids2011.geojson')[['lsoa01cd', 'geometry']]

# base.to_file("tmp/OA11.geojson", driver="GeoJSON")
# centroids.to_file("tmp/centroids11.geojson", driver="GeoJSON")

# os.system('tippecanoe -Z6 -z9 -f -o tiles/area_centroids_6_9.mbtiles --coalesce-densest-as-needed --generate-id ./tmp/OA%d.geojson ./tmp/centroids%d.geojson'%(11,11))




In [63]:
# merge
os.system('tile-join -o ./tiles/merged.mbtiles -f ./tiles/countries.mbtiles ./tiles/area_centroids_6_9.mbtiles')

Tile 6/31/20 size is 517461, >500000. Skipping this tile
.Tile 6/31/21 size is 539867, >500000. Skipping this tile


0

In [41]:
# tippecanoe -z3 -o countries-z3.mbtiles --coalesce-densest-as-needed ne_10m_admin_0_countries.geojson
# tippecanoe -zg -Z4 -o states-Z4.mbtiles --coalesce-densest-as-needed --extend-zooms-if-still-dropping ne_10m_admin_1_states_provinces.geojson
# tile-join -o states-countries.mbtiles countries-z3.mbtiles states-Z4.mbtiles

SyntaxError: invalid syntax (1242424097.py, line 1)

In [68]:
## convert to static server

os.system('rm -rf ./tiles/tileserver; mb-util --image_format=pbf ./tiles/merged.mbtiles ./tiles/tileserver')
os.system('cd ./tiles/tileserver && find . -type f | xargs -n1 -P 1 -t -I % gzip -d -r -S .pbf %')
# os.system("cd ./tiles/tileserver && find . -type f -exec mv '{}' '{}'.pbf \;")

DEBUG:mbutil.util:Exporting MBTiles to disk
DEBUG:mbutil.util:./tiles/merged.mbtiles --> ./tiles/tileserver
DEBUG:mbutil.util:flipping
INFO:mbutil.util:1 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:2 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:3 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:4 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:5 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:6 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:7 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:8 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:9 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:10 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:11 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:12 / 177 tiles exported
DEBUG:mbutil.util:flipping
INFO:mbutil.util:13 / 177 tiles exported
DEBUG:mbutil.util

gzip -d -r -S .pbf ./9/250/171.pbf
gzip -d -r -S .pbf ./9/250/165.pbf
gzip -d -r -S .pbf ./9/250/170.pbf
gzip -d -r -S .pbf ./9/250/166.pbf
gzip -d -r -S .pbf ./9/250/172.pbf
gzip -d -r -S .pbf ./9/250/173.pbf
gzip -d -r -S .pbf ./9/250/167.pbf
gzip -d -r -S .pbf ./9/258/169.pbf
gzip -d -r -S .pbf ./9/258/168.pbf
gzip -d -r -S .pbf ./9/258/171.pbf
gzip -d -r -S .pbf ./9/258/170.pbf
gzip -d -r -S .pbf ./9/258/167.pbf
gzip -d -r -S .pbf ./9/251/169.pbf
gzip -d -r -S .pbf ./9/251/168.pbf
gzip -d -r -S .pbf ./9/251/163.pbf
gzip -d -r -S .pbf ./9/251/162.pbf
gzip -d -r -S .pbf ./9/251/161.pbf
gzip -d -r -S .pbf ./9/251/171.pbf
gzip -d -r -S .pbf ./9/251/165.pbf
gzip -d -r -S .pbf ./9/251/164.pbf
gzip -d -r -S .pbf ./9/251/170.pbf
gzip -d -r -S .pbf ./9/251/166.pbf
gzip -d -r -S .pbf ./9/251/172.pbf
gzip -d -r -S .pbf ./9/251/167.pbf
gzip -d -r -S .pbf ./9/256/169.pbf
gzip -d -r -S .pbf ./9/256/168.pbf
gzip -d -r -S .pbf ./9/256/171.pbf
gzip -d -r -S .pbf ./9/256/165.pbf
gzip -d -r -S .pbf .

256